# This is for collecting small corpus on Baidubaike 

In [1]:
#Source code from ： https://github.com/newer-plus/baidubaike-corpus 

In [1]:
from pymongo import MongoClient

In [2]:
#connect to mongo service
client = MongoClient('put your mongo link here')
client.list_database_names()

['baidubaike', 'admin', 'local']

In [ ]:
## import requests as rq
import re
import datetime
import time
from urllib.parse import unquote, urlparse, urlunparse
import pymongo
from html import unescape
from multiprocessing import Pool as ProcessPool
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Value
from bs4 import BeautifulSoup
from pymongo import MongoClient
import defs


client = MongoClient('your mongo link')
client.list_database_names()
#creat a database
db = client['baidubaike']
#create a collection
tasks = db['tasks']
items = db['items']
#pymongo.MongoClient().drop_database('baidubaike')
#tasks = pymongo.MongoClient().baidubaike.tasks  # same the task into database
#items = pymongo.MongoClient().baidubaike.items  # save the result
tasks.create_index([('url', 'hashed')])  # create a index to follow the process
items.create_index([('url', 'hashed')])
count = items.count()  # the numbers of pages that have already collected
num = Value('L',count)
if tasks.count() == 0:  # here i started from three website of entries (literature, movie and series)
    url1 = 'http://baike.baidu.com/item/文学'
    url2 = 'http://baike.baidu.com/item/电影'
    url3 = 'http://baike.baidu.com/item/电视剧'
    d1 = {'url': url1}
    d2 = {'url': url2}
    d3 = {'url': url3}
#    if not items.find({'url': {'$in': [url1, url2, url3]}}):
    tasks.insert_many([d1, d2, d3])

url_split_re = re.compile('[&+]')


def clean_url(url):
    url = urlparse(url)
    return url_split_re.split(urlunparse((url.scheme, url.netloc, url.path, '', '', '')))[0]


#this can apply to the baidu head
header = {
    "User-Agent":
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/81.0.4044.122 Safari/537.36'}



def run():
    def main():
        while tasks.count() > 0:
            url = tasks.find_one_and_delete({})['url']  # Get a url and delete it in the queue
            try:
                sess = rq.get(url, headers=header)
            except:
                continue
            web = sess.content.decode('utf-8', 'ignore')
            urls = re.findall('href="(/item/.*?)"', web)  # Find all in site links
            for u in urls:
                try:
                    u = unquote(str(u))
                except:
                    continue
                u = 'http://baike.baidu.com' + u
                u = clean_url(u)
                if not items.find_one({'url': u}):  # Add links that have not been queued to the queue
                    tasks.update({'url': u}, {'$set': {'url': u}}, upsert=True)
            soup = BeautifulSoup(web)
            text = soup.find_all('div', class_='para')
            # To crawl the information we need, we need regular expression knowledge to write according to the web page source code
            if text:
                text = ' '.join(
                    [re.sub('[ \n\r\t\u3000]+', ' ', re.sub('<.*?>|\xa0', '', unescape(str(t))).strip()) for t in
                     text])  # Do some simple treatment to the results of crawling
                title = re.findall(u'<title>(.*?)_百度百科</title>', str(soup.title))[0]
                items.update({'url': url}, {'$set': {'url': url, 'title': title, 'text': text}}, upsert=True)
                num.value +=1
                print('%s, 爬取《%s》，URL: %s, 已经爬取%s' % (datetime.datetime.now(), title, url, num.value))

                
    p = ThreadPool(4, defs.main)  # 多线程爬取，4是线程数
    p.close()
    p.join()


if __name__ == '__main__':
    pool = ProcessPool(4, defs.run)  # 多进程爬取，4是线程数
    pool.close()
    pool.join()
    time.sleep(30)
    pool.terminate()
    

